In [18]:
class ForwardChainingFOL:
    def __init__(self):
        self.facts = set()
        self.rules = []

    def add_fact(self, fact):
        self.facts.add(fact)

    def add_rule(self, premises, conclusion):
        self.rules.append((premises, conclusion))

    def unify(self, fact, premise):
        if "(" not in premise:
            return fact == premise
        if premise.split("(")[0] != fact.split("(")[0]:
            return None

        fact_args = fact[fact.index("(") + 1 : -1].split(",")
        premise_args = premise[premise.index("(") + 1 : -1].split(",")
        if len(fact_args) != len(premise_args):
            return None

        substitution = {}
        for f_arg, p_arg in zip(fact_args, premise_args):
            if p_arg.islower():
                if p_arg in substitution:
                    if substitution[p_arg] != f_arg:
                        return None
                else:
                    substitution[p_arg] = f_arg
            elif f_arg != p_arg:
                return None
        return substitution

    def apply_substitution(self, expr, substitution):
        if "(" not in expr:
            return substitution.get(expr, expr)

        predicate = expr.split("(")[0]
        args = expr[expr.index("(") + 1 : -1].split(",")
        substituted_args = [substitution.get(arg, arg) for arg in args]
        return f"{predicate}({','.join(substituted_args)})"

    def infer(self):
        new_inferences = True
        while new_inferences:
            new_inferences = False
            for premises, conclusion in self.rules:
                substitutions = []
                for premise in premises:
                    for fact in self.facts:
                        subst = self.unify(fact, premise)
                        if subst is not None:
                            substitutions.append(subst)
                            break

                if len(substitutions) == len(premises):
                    combined_subst = {}
                    for subst in substitutions:
                        combined_subst.update(subst)

                    inferred_fact = self.apply_substitution(conclusion, combined_subst)
                    if inferred_fact not in self.facts:
                        print(f"Inferred: {inferred_fact}")
                        self.facts.add(inferred_fact)
                        new_inferences = True

    def query(self, query_fact):
        if query_fact in self.facts:
            print(f"Proved: {query_fact}")
            return True
        else:
            print(f"Could not prove: {query_fact}")
            return False


kb = ForwardChainingFOL()

kb.add_fact("American(Robert)")
kb.add_fact("Missile(T1)")
kb.add_fact("Owns(A,T1)")
kb.add_fact("Enemy(A,America)")

kb.add_rule(["Missile(x)", "Owns(A,x)"], "Sells(Robert,x,A)")
kb.add_rule(["Missile(x)"], "Weapon(x)")
kb.add_rule(["Enemy(x,America)"], "Hostile(x)")
kb.add_rule(["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)")

kb.infer()
kb.query("Criminal(Robert)")


Inferred: Sells(Robert,T1,A)
Inferred: Weapon(T1)
Inferred: Hostile(A)
Inferred: Criminal(Robert)
Proved: Criminal(Robert)


True